In [27]:
import json
import random
import time
import importlib

from Project.utils.api import truth_api as api
from Project.utils.storage import truth_db as db
from Project.utils.analysis import text_classifier as tc

importlib.reload(api)
importlib.reload(db)
importlib.reload(tc)


<module 'Project.utils.analysis.text_classifier' from 'C:\\Users\\Filippo Corti\\Documents\\GitHub\\SocialMediaMining\\Project\\utils\\analysis\\text_classifier.py'>

In [28]:
credentials = json.load(open('../../keys/truth_key.json'))

username = credentials['username']
password = credentials['password']

In [29]:
truth = api.TruthApi(username, password)
truth_db = db.SQLiteTruthSaver(db_name='../db/truthsocial.db')

In [ ]:
# 1. Extract the last 500 posts about tariffs
idx = 0
for post in truth.search_posts(keyword="tariffs", limit=500):
    print(f"{idx}: New post: {post.id}")
    truth_db.insert_post(post)
    idx+=1

In [ ]:
# 2. Extract the comments of each of the posts
posts = truth_db.get_all_post_ids()

for idx, post_id in enumerate(posts):
    post_details = truth_db.get_post_by_id(post_id)
    print(f"{idx}: Extracting {post_details.replies_count} comments from post: {post_id}")
    for post in truth.get_comments(post_id):
        truth_db.insert_post(post)

    if post_details.replies_count > 2:
        sleep_time = random.randrange(start=15, stop=60)
        time.sleep(sleep_time)


In [30]:
text_classifier = tc.TextClassifier()

all_posts = truth_db.get_all_posts()

In [31]:
all_posts = [
    post
    for post in all_posts
]

random.shuffle(all_posts)

In [32]:
print(len(all_posts))

72375


In [ ]:
current_batch = list() # Technically not a batch

for idx, post in enumerate(all_posts):

    current_batch.append(post)

    if len(current_batch) < 5 and idx != len(all_posts) - 1:
        continue

    gemini_labels = text_classifier.get_gemini_bt_stance(
        texts=[comment.content for comment in current_batch]
    )

    for i in range(len(current_batch)):
        comment_id = current_batch[i].id
        gemini_label = gemini_labels[i]

        truth_db.insert_comment_analysis(
            comment_id=comment_id,
            gemini_label=gemini_label,
        )

    current_batch = list()

    idx += 1
    if idx % 100 == 0:
        print(f"{idx}/{len(all_posts)}")

In [5]:
truth_db.close()